Mattie Gisselbeck

GIS 5572: Spatial Data Science II
    
Lab 1.2

In [77]:
import psycopg2
import requests
import json
import os
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

In [ ]:
# Sets Environment 
arcpy.env.workspace = "\\Mac\Home\Documents\ArcGIS\Projects\Lab1_2"
wksp = arcpy.env.workspace

1.2.1: Creates a Polygon using ArcPy Geometry Primitives 

In [20]:
# Creates List of Points
point_list = [arcpy.Point(1, 1), arcpy.Point(1, 2), arcpy.Point(2, 2), arcpy.Point(2, 1), arcpy.Point(1, 1)]

# Creates Polygon from the Array
polygon = arcpy.Polygon(arcpy.Array(point_list))

1.2.2: Converts Polygon to .WKT

In [21]:
wkt = polygon.WKT

1.2.3: Loads Polygon into Postgres DB Using psycopg2

In [30]:
# Connects to PostgresSQL Database Using psycopg2
connection = psycopg2.connect(host = '34.133.121.12',
                              database = 'lab1',
                              user = 'postgres',
                              password = 'student',
                             )
cursor = connection.cursor()

# Creates and populates table in the PostgresSQL Database
cursor.execute("CREATE TABLE IF NOT EXISTS polygon_table (id SERIAL PRIMARY KEY, geom GEOMETRY)")
cursor.execute("INSERT INTO polygon_table (geom) VALUES (ST_GeomFromText('{}', 4326))".format(polygon.WKT))
connection.commit()
connection.close()

1.2.3: get_polygon Application Using Flask

In [ ]:
# Installs PostGIS, psycopg2, and Flask in SSH
# Note: This is in the VM Instance SSH-in-browser in Google Cloud. 
sudo apt-get update
sudo apt-get install postgis
pip install psycopg2-binary
pip install Flask

In [66]:
# Creates get_polygon application and posts to SSH using Flask
# Note: This is in the VM Instance SSH-in-browser in Google Cloud. 

import psycopg2
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/', methods=['GET']) 
def get_geojson():
    reponse = get_polygon()
    return response

def get_polygon():
cursor = connection.cursor()
cursor.execute("SELECT ST_AsGeoJSON(location) FROM polygon_table;") result = cursor.fetchone()
if result is None:
return jsonify({'error': 'Polygon not found'}), 404
    else:
        return jsonify({'geojson': result[0]})
    
if __name__ == "__main__":
app.run(debug=True, host='0.0.0.0', port=5000)

@app.route('/print_data', methods=['POST'])
def print_data():
print("*********************")
print("*********************")
print(request.method) 
print(request.data) 
print("*********************")
print("*********************")
return "Accepted 202 - post received; printed to console"

1.2.4: Retrieving polygon.geojson via Google Cloud VM Instance API 

In [69]:
# Retrives polygon.geojson from Google Cloud VM Instance API
api = r'http://34.29.175.222:5000/'
dictionary = json.loads(requests.get(api).text)

# Corrects formatting by removing a pair of brackets
sub = dictionary['geojson'].replace("[[[[", "[[[").replace("]]]]", "]]]")

# Converts string into a JSON dictionary
json_dictionary = json.loads(sub)
json_dictionary

{'type': 'MultiPolygon', 'coordinates': [[[1.00012207, 1.00012207], [2.00012207, 1.00012207], [2.00012207, 2.00012207], [1.00012207, 2.00012207], [1.00012207, 1.00012207]]]}

In [ ]:
# Creates a GeoJSON dictionary
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": dictionary_2['coordinates'],
        "type": "Polygon"
      }
    }
  ]
}

# Sets path file for the GeoJSON dictionary
path = os.path.join(wksp, 'geojson.json')
with open (path, 'w') as json_file:
    json.dump(geojson, json_file)

1.2.5: Publishes polygon.geojson to ArcGIS Online

In [ ]:
# Signs into ArcGIS Online
username = 'mgisselbeck'
password = 'Jumponit98'
gis = GIS('https://www.arcgis.com', username, password)

# Publishes polygon.geojson as a Feature Layer to ArcGIS Online
item = gis.content.add({'type': 'GeoJson'}, 'geojson.json')
feature_layer_item = item.publish()
feature_layer_url = feature_layer_item.url